In [3]:
import vaex
import vaex.jupyter
import numpy as np
#import pylab as plt
import bqplot.pyplot as plt
import subprocess

In [4]:
def run_process(args):
    print('='*50)
    print(' '.join(args))
    print('-'*50)
    completed_process = subprocess.run(args,capture_output=True)
    assert completed_process.returncode == 0
    print(completed_process.stdout.decode('utf-8'))

In [5]:
%%time

FILENAME = 'test.cpp'    

run_process(['cat',
             FILENAME])

run_process(['g++',
             FILENAME])
    
run_process(['/setup/pintool/pin',
             '-t', '/setup/pintool/source/tools/ManualExamples/obj-intel64/pinatrace.so',
             '--', './a.out'])

run_process(['wc',
             'pinatrace.out'])

print('='*50)

cat test2.cpp
--------------------------------------------------
#include <bits/stdc++.h>
using namespace std;

#define SIZE (int)1e2
#define SIZE2 (int)1e4
int main() {
	vector<int> arr(SIZE);
	for(int i = 0; i < SIZE; i++) 
		for(int j = 0; j < SIZE2; j++) 
			arr[j] = j;
	
	return 0;
}

g++ test2.cpp
--------------------------------------------------

/setup/pintool/pin -t /setup/pintool/source/tools/ManualExamples/obj-intel64/pinatrace.so -- ./a.out
--------------------------------------------------

wc pinatrace.out
--------------------------------------------------
 15642591  46927771 516205475 pinatrace.out

CPU times: user 12.6 ms, sys: 19 ms, total: 31.7 ms
Wall time: 4min 2s


In [6]:
%%time
df = vaex.from_csv('pinatrace.out', sep=' ', names=['inst_addr', 'RW', 'mem_addr'], header=None)
df.export_hdf5('pinatrace.hdf5')

CPU times: user 3min 26s, sys: 2.43 s, total: 3min 28s
Wall time: 3min 44s


In [5]:
%%time
df = vaex.open('pinatrace.hdf5')[:-1]

CPU times: user 9.77 ms, sys: 0 ns, total: 9.77 ms
Wall time: 14.8 ms


In [6]:
df['mem_addr_int'] = df.mem_addr.apply(lambda x: int(x,16))
print('# lines: %d' % len(df))
df.head()

# lines: 635644


#,inst_addr,RW,mem_addr,index,mem_addr_int
<i style='opacity: 0.6'>0</i>,0x7fee28191093:,W,0x7ffdeeb63fe8,0,1.40729e+14
<i style='opacity: 0.6'>1</i>,0x7fee28191ea0:,W,0x7ffdeeb63fe0,1,1.40729e+14
<i style='opacity: 0.6'>2</i>,0x7fee28191ea4:,W,0x7ffdeeb63fd8,2,1.40729e+14
<i style='opacity: 0.6'>3</i>,0x7fee28191ea6:,W,0x7ffdeeb63fd0,3,1.40729e+14
<i style='opacity: 0.6'>4</i>,0x7fee28191ea8:,W,0x7ffdeeb63fc8,4,1.40729e+14
<i style='opacity: 0.6'>5</i>,0x7fee28191eaa:,W,0x7ffdeeb63fc0,5,1.40729e+14
<i style='opacity: 0.6'>6</i>,0x7fee28191eaf:,W,0x7ffdeeb63fb8,6,1.40729e+14
<i style='opacity: 0.6'>7</i>,0x7fee28191ebf:,R,0x7fee283b7e68,7,1.40661e+14
<i style='opacity: 0.6'>8</i>,0x7fee28191ec6:,W,0x7fee283b7720,8,1.40661e+14
<i style='opacity: 0.6'>9</i>,0x7fee28191ed7:,R,0x7fee283b8000,9,1.40661e+14


In [7]:
df.plot_widget(df.index, df.mem_addr_int, f='log1p', backend='bqplot')

PlotTemplate(components={'main-widget': VBox(children=(VBox(children=(Figure(axes=[Axis(color='#666', grid_col…

Plot2dDefault(w=None, what='count(*)', x='index', y='mem_addr_int', z=None)

In [9]:
%%time
fig = plt.figure()
plt.scatter(df.index.values, df.mem_addr_int.values, default_size=2)
plt.show()

CPU times: user 42.2 s, sys: 1.64 s, total: 43.8 s
Wall time: 47.8 s


In [ ]:
%%time
fig = plt.figure()
plt.scatter(df.index.values[700000:-7000], df.mem_addr_int.values[700000:-7000], default_size=2, selection=[df.mem_addr_int>int(4e8)])
plt.show()